In [204]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import mysql.connector as conn
import statsmodels.api as sm

import re
import json
import glob
import warnings
warnings.filterwarnings('ignore')

In [205]:
username = None
password = None

with open("../connection.json") as connection_file:
    cf = json.load(connection_file)
    username = cf["user"]
    password = cf["password"]

cnx = conn.connect(
    host="localhost",
    database="foodaps",
    user=username,
    password=password,
)

In [206]:
AME_QUERY_FILE = "../sql/ame.sql"
ENU_QUERY_FILE = "../sql/enu.sql"
INDIV_CHAR_QUERY_FILE = "../sql/indiv_char.sql"
HOUSEHOLD_CHAR_QUERY_FILE = "../sql/household_char.sql"
FAH_NUTRIENTS_QUERY_FILE = "../sql/fah_nutrients.sql"

enu_weights = {
    "breakfast": 0.16, 
    "lunch": 0.43, 
    "dinner": 0.30, 
    "snack": 0.11
}

def get_query(conn, query_file): 
    """
    Given a query and a connection returns a pandas table. 
    """
    with open(query_file, "r") as f:
        query = f.read()
    return pd.read_sql(query, conn)

def create_enu_table(): 
    """
    Creates a table from two queries to calculate the 
    equivalent nutritional units (ENU) for each person in the
    houshold and aggregates it to the household level. Does 
    not account for guests. 
    """
    ame = get_query(cnx, AME_QUERY_FILE)
    enu = get_query(cnx, ENU_QUERY_FILE)

    enu["tot_snacks"] = enu["tot_snackam"] + enu["tot_snackpm"]
    enu["weekly_enu"] = (enu["tot_brkfst"]*enu_weights["breakfast"] 
        + enu["tot_lunch"]*enu_weights["lunch"] 
        + enu["tot_dinner"]*enu_weights["dinner"] 
        + enu["tot_snacks"]*enu_weights["snack"])

    return ame.merge(enu, left_on=["HHNUM", "PNUM"], right_on=["HHNUM", "PNUM"], how="left").groupby(
        "HHNUM").agg({
            "weekly_enu": "sum",
            "TEE": "sum", 
            "AME": "sum"
        }).reset_index()

def majority_vote_household_demographic(hh_series):
    """
    If a household comprises of members of the same 
    race, then returns that race. Otherwise, returns
    "mixed".
    """
    if hh_series.unique().shape[0] == 1:
        return hh_series.unique()[0]
    else:
        return "mixed"

def create_household_table():
    """
    Creates a table from two queries to create household
    characteristics for explanatory analysis. 
    """
    indiv = get_query(cnx, INDIV_CHAR_QUERY_FILE)
    hh = get_query(cnx, HOUSEHOLD_CHAR_QUERY_FILE)

    hh["guest_enu"] = (hh["nguest_brkfst"] * enu_weights["breakfast"] + 
        hh["nguest_lunch"] * enu_weights["lunch"] + 
        hh["nguest_dinner"] * enu_weights["dinner"] + 
        hh["nguest_snack"] * enu_weights["snack"])

    hh_demo = indiv.groupby("hhnum").agg({
        "Race": majority_vote_household_demographic,
        "Hispanic": majority_vote_household_demographic,
        "female_head": "first"}).reset_index()

    return hh.merge(hh_demo, left_on="hhnum", right_on="hhnum", how="left")

def create_nutrients_table(): 
    """
    Creates a table from a query for weekly nutrition by household. 
    """
    fah_nut = get_query(cnx, FAH_NUTRIENTS_QUERY_FILE)
    fah_nut = fah_nut.dropna()

    #remove SUM() and hh_ from column names
    fah_nut.columns = [col.split("_")[1] if "SUM" in col else col for col in fah_nut.columns]

    #remove ) from column names
    fah_nut.columns = [col.split(")")[0] if ")" in col else col for col in fah_nut.columns]

    #add fah to column_names except for hhnum
    fah_nut.columns = ["fah_" + col if col != "hhnum" else col for col in fah_nut.columns]

    return fah_nut

def create_dataset(): 
    enu = create_enu_table()
    hh = create_household_table()
    nut = create_nutrients_table()
    snap = nut.merge(enu, left_on="hhnum", right_on="HHNUM", how="left").merge(
    hh, left_on="hhnum", right_on="hhnum", how="left")

    return snap

def prepare_data():
    snap = create_dataset()
    snap["total_enus"] = snap["weekly_enu"] + snap["guest_enu"]
    snap["total_enus"] = snap["total_enus"] * snap["AME"]

    # drop rows with total_enu == 0, meaning no meals at home were eaten
    snap = snap[snap["total_enus"] != 0]

    # engineer explanatory variables
    snap["household_recieves_snap"] = (snap["snapnowhh"] > 0).astype("int")
    snap["household_weekly_snap_benefits"] = snap["SNAPLASTAMT"]/snap["AME"]
    snap["household_weekly_money_income"] = snap["inchhavg_r"]/snap["AME"]
    snap["household_weekly_money_income_squared"] = snap["household_weekly_money_income"]**2
    snap["school_breakfast"] = (snap["schservebrkfst"] > 0).astype("int")
    snap["houshold_recieves_wic"] = (snap["wichh"] > 0).astype("int")
    snap["female_head_present"] = (snap["female_head"])
    snap["black"] = (snap.Race == "Black").astype("int")
    snap["hispanic"] = (snap.Hispanic == "Hispanic").astype("int")
    snap["guest_meals"] = snap["guest_enu"] / snap["AME"]
    return snap 

In [207]:
snap = prepare_data()

explanatory_vars_mpc = [
    "household_weekly_snap_benefits",
    "household_weekly_money_income",
    "household_weekly_money_income_squared",
    "school_breakfast",
    "houshold_recieves_wic",
    "female_head_present",
    "black",
    "hispanic",
    "midwest",
    "south", 
    "west",
    "guest_meals",
    "nonmetro"
]

explanatory_vars_binary = [
    "household_recieves_snap",
    "household_weekly_money_income",
    "household_weekly_money_income_squared",
    "school_breakfast",
    "houshold_recieves_wic",
    "female_head_present",
    "black",
    "hispanic",
    "midwest",
    "south", 
    "west",
    "guest_meals",
    "nonmetro"
]

response_vars = [
    nut_var for nut_var in snap.columns if re.match(r"fah_*", nut_var)
]

In [208]:
## dataset restriction 1 - target group 1 & 4
restriction_1 = snap[snap["targetgroup"].isin([1,4])]

## dataset restriction 2 - target group 1, 2, & 4
restriction_2 = snap[snap["targetgroup"].isin([1,2,4])]

## dataset restriction 3 - target group 2 & 4
restriction_3 = snap[snap["targetgroup"].isin([2,4])]

def star_rating(p_value):
    """
    Using bonferroni correction, returns a star rating for the p-value. 
    """
    if p_value < 0.001/len(response_vars):
        return "***"
    elif p_value < 0.01/len(response_vars):
        return "**"
    elif p_value < 0.05/len(response_vars):
        return "*"
    else:
        return ""

def run_results(explanatory_vars, response_vars, restricted_dataset):
    ## run through regression for each nutrient and save results to a log file
    snap_var = explanatory_vars[0]
    X = restricted_dataset[explanatory_vars]
    X = sm.add_constant(X)

    for nut_var in response_vars:
        y = restricted_dataset[nut_var]/restricted_dataset["AME"]
        model = sm.OLS(y, X).fit(cov_type="HC2")

        # print out the response variable the coefficent for the household_weekly_snap_benefits variable
        # the standard error, and the p-value for the household_recieves_snap variable, if the 
        # p-value is less than 0.05 print one star, if less than 0.01 print two stars, if less than
        # 0.001 print three stars

        print(f"{nut_var} \t {model.params[snap_var]:.4f}"
            f"({model.bse[snap_var]:.4f})  \t" 
            f"{model.pvalues[snap_var]:.5f}"
            f"{star_rating(model.pvalues[snap_var])}")

run_results(explanatory_vars_mpc, response_vars, restriction_1)

fah_energy 	 22.8739(6.2684)  	0.00026**
fah_carb 	 2.9461(0.8260)  	0.00036**
fah_dietfiber 	 0.0813(0.0402)  	0.04303
fah_totsug 	 1.6653(0.4382)  	0.00014**
fah_totfat 	 0.9418(0.2978)  	0.00156*
fah_satfat 	 0.3406(0.0966)  	0.00042**
fah_monofat 	 0.3299(0.1122)  	0.00329
fah_polyfat 	 0.1928(0.0853)  	0.02388
fah_protein 	 0.7383(0.2065)  	0.00035**
fah_chol 	 3.0143(0.8947)  	0.00075*
fah_sodium 	 34.6854(23.3137)  	0.13681
fah_vitarae 	 5.5131(2.2494)  	0.01425
fah_vitb6 	 0.0126(0.0059)  	0.03387
fah_vitb12 	 0.0649(0.0218)  	0.00288
fah_vitc 	 0.5099(0.2033)  	0.01215
fah_iron 	 0.1265(0.0501)  	0.01156
fah_thiamin 	 0.0123(0.0051)  	0.01637
fah_riboflavin 	 0.0185(0.0058)  	0.00145*
fah_calcium 	 10.5989(3.0741)  	0.00057*
fah_phosphorus 	 13.7318(3.9435)  	0.00050*
fah_magnes 	 2.2126(0.6733)  	0.00102*


In [209]:
run_results(explanatory_vars_mpc, response_vars, restriction_2)

fah_energy 	 21.3486(5.4147)  	0.00008**
fah_carb 	 2.6242(0.7060)  	0.00020**
fah_dietfiber 	 0.0567(0.0355)  	0.10959
fah_totsug 	 1.5523(0.3743)  	0.00003***
fah_totfat 	 0.9048(0.2613)  	0.00054*
fah_satfat 	 0.3102(0.0809)  	0.00013**
fah_monofat 	 0.3335(0.0985)  	0.00071*
fah_polyfat 	 0.1850(0.0794)  	0.01986
fah_protein 	 0.6903(0.1794)  	0.00012**
fah_chol 	 2.8034(0.8025)  	0.00048*
fah_sodium 	 40.9713(18.8170)  	0.02945
fah_vitarae 	 6.0652(2.2681)  	0.00749
fah_vitb6 	 0.0139(0.0050)  	0.00548
fah_vitb12 	 0.0713(0.0215)  	0.00089*
fah_vitc 	 0.3905(0.1797)  	0.02978
fah_iron 	 0.1117(0.0424)  	0.00848
fah_thiamin 	 0.0110(0.0043)  	0.01031
fah_riboflavin 	 0.0171(0.0049)  	0.00050*
fah_calcium 	 9.0821(2.5836)  	0.00044**
fah_phosphorus 	 12.1350(3.3390)  	0.00028**
fah_magnes 	 1.7997(0.5931)  	0.00241


In [210]:
run_results(explanatory_vars_mpc, response_vars, restriction_3)

fah_energy 	 24.3356(5.8393)  	0.00003***
fah_carb 	 2.9870(0.7602)  	0.00009**
fah_dietfiber 	 0.0728(0.0386)  	0.05914
fah_totsug 	 1.7679(0.3972)  	0.00001***
fah_totfat 	 1.0416(0.2847)  	0.00025**
fah_satfat 	 0.3525(0.0864)  	0.00004***
fah_monofat 	 0.3963(0.1072)  	0.00022**
fah_polyfat 	 0.2087(0.0892)  	0.01924
fah_protein 	 0.7429(0.1907)  	0.00010**
fah_chol 	 3.0195(0.8607)  	0.00045**
fah_sodium 	 42.2878(20.0170)  	0.03464
fah_vitarae 	 7.1458(2.4781)  	0.00393
fah_vitb6 	 0.0152(0.0054)  	0.00448
fah_vitb12 	 0.0741(0.0232)  	0.00139*
fah_vitc 	 0.5122(0.1933)  	0.00807
fah_iron 	 0.1274(0.0465)  	0.00611
fah_thiamin 	 0.0125(0.0047)  	0.00755
fah_riboflavin 	 0.0190(0.0053)  	0.00036**
fah_calcium 	 10.2865(2.7748)  	0.00021**
fah_phosphorus 	 13.2601(3.5859)  	0.00022**
fah_magnes 	 2.0882(0.6349)  	0.00101*


In [211]:
cnx.close()